In [2]:
import os
from openpyxl import load_workbook
import pandas as pd
from itertools import chain
import re


In [3]:
to_exclude = 'load,rename,recode,refilter,df,title'.split(',') 

In [4]:

# Get the current directory
current_directory = os.getcwd()

# Define the name of the Excel file you want to open
file_name = "recode.xlsx"

# Construct the full path to the Excel file
file_path = os.path.join(current_directory, file_name)


df_original_ = pd.read_excel(file_path, sheet_name="original")
df_recode_ = pd.read_excel(file_path, sheet_name="recode")
df_analysis_ = pd.read_excel(file_path, sheet_name="analysis")


In [5]:
df_recode = pd.concat([df_original_, df_recode_], axis=0).dropna(subset=['nvar'])

In [22]:
df_analysis = df_analysis_[(df_analysis_['include']==1) & (~df_analysis_['command'].isin(to_exclude))]
# col_2_vars = df_analysis[[2]][2].str.extract(r'var:(.+),')[0].unique()
col_2_vars_ = []
if not df_analysis[[2]][2].dropna().empty:
  col_2_vars_1 = df_analysis[[2]][2].str.findall(r'[#\$]([\w_]+)').dropna()
  col_2_vars_2 = df_analysis[[2]][2].str.findall(r'var:(.+),').dropna()
  col_2_vars_ = col_2_vars_1 + col_2_vars_2
col_2_vars = set(list(chain.from_iterable(col_2_vars_))) # set to get unique, chain to flatten

col_1_vars_ = []
if not df_analysis[[1]][1].dropna().empty:
  col_1_vars_ = df_analysis[[1]][1].str.findall(r'[#\$]([\w_]+)').dropna()
col_1_vars = set(list(chain.from_iterable(col_1_vars_))) # set to get unique, chain to flatten

vars_list = set(list(col_2_vars) + list(col_1_vars))
vars_list = [x for x in vars_list if pd.notna(x)]
vars_list = sorted([x for x in vars_list])

df_var_list = pd.DataFrame(vars_list,columns = ['var'])
df_var_list['definition'] = ''

In [6]:
def replace_pattern(match):
    # pattern = match.group(0)  # Get the matched pattern
    num = match.group(1) if match.group(1) is not None else match.group(2)
    # print(num)
    return "$" +df_recode.loc[df_recode['id'] == int(num), 'nvar'].iloc[0]  # Replace with corresponding 'nvar' value


In [185]:
new_fx = []
for value in df_recode['fx']:
  # print(i,r['fx'])  
  if not pd.isna(value):
    new_fx.append(re.sub(r'[#\$](\d+)', replace_pattern, value))
  else:
    new_fx.append(value)
df_recode['new_fx'] = new_fx

In [236]:

def iterative_build_fx(nvar,built,level):
  # get formula for the nvar
  fx =  df_recode[df_recode['nvar']==nvar]['new_fx'].values[0]
  var = df_recode[df_recode['nvar']==nvar]['var'].values[0]
  if pd.isna(fx):
    built = built + '\t' * level + nvar + '=' + var + '\n'
    return built
  fx_str = str(fx) if not isinstance(fx, str) else fx
  # add formula to built using level as tabs
  built = built + '\t' * level + nvar + '=' + fx_str + '\n'
  # get list of each nvar in the formula
  nvars = [x for x in  set(re.findall(r'[#\$]([\w_]+)',fx))]
  for nvar in nvars:
    built = iterative_build_fx(nvar,built,level+1)
  return built

for nvar in vars_list:
  df_var_list.loc[df_var_list['var'] == nvar,'definition'] = iterative_build_fx(nvar, '', 0) 
 
  

In [238]:
print(df_var_list)

                      var                                         definition
0                     age                                          age=AGE\n
1                    dm10         dm10=iif($dm>0,1,0)\n\tdm=PREOP_DIABETES\n
2      evt_days_gp5_for_r  evt_days_gp5_for_r=#evt_days_gp5\n\tevt_days_g...
3               evt_for_r             evt_for_r=an$evt\n\tevt=POSTOP_NEURO\n
4   evt_ret_tia_gp5_for_r  evt_ret_tia_gp5_for_r=switch(var:#evt_ret_tia_...
5       evt_stk_gp5_for_r  evt_stk_gp5_for_r=switch(var:#evt_stk_30,<1:1,...
6      evt_stk_mi_death_r  evt_stk_mi_death_r=#evt_stk_mi_death_\n\tevt_s...
7            evt_stk_mi_r  evt_stk_mi_r=#evt_stk_mi_\n\tevt_stk_mi_=iif(#...
8               evt_stk_r  evt_stk_r=iif($evt_no_stk==1,0,1)\n\tevt_no_st...
9       evt_tia_gp5_for_r  evt_tia_gp5_for_r=switch(var:#evt_tia_30,<1:1,...
10                  htn10                 htn10=iif($htn>0,1,0)\n\thtn=HTN\n
11                  race3  race3=iif($race==5,1,iif($race==3,2,3))\n\trac...